In [1]:
# Imports

########################################################################
# Python Standard Libraries
import os
import multiprocessing
from timeit import default_timer as timer

########################################################################
# Numpy Library
import numpy as np # linear algebra

########################################################################
# Pandas Library
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

########################################################################
# MATPLOT Library
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.ticker import MaxNLocator
%matplotlib inline

########################################################################
# SKLearn Library
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_recall_curve, classification_report, confusion_matrix, average_precision_score, roc_curve, auc, multilabel_confusion_matrix

########################################################################
# SCIPY Library
from scipy.stats import gaussian_kde
import scipy.stats as st

In [2]:
# Utility functions
########################################################################
# Print system information
def print_system_info():
    mem_bytes = os.sysconf('SC_PAGE_SIZE') * os.sysconf('SC_PHYS_PAGES')  # e.g. 4015976448
    mem_gib = mem_bytes/(1024.**3)  # e.g. 3.74
    print("{:<23}{:f} GB".format('RAM:', mem_gib))
    print("{:<23}{:d}".format('CORES:', multiprocessing.cpu_count()))
    !lscpu

########################################################################
# Walk through input files
def print_input_files():
    # Input data files are available in the "../input/" directory.
    for dirname, _, filenames in os.walk('/kaggle/input'):
        for filename in filenames:
            print(os.path.join(dirname, filename))

########################################################################
# Dump text files
def dump_text_file(fname):
    with open(fname, 'r') as f:
        print(f.read())

########################################################################
# Dump CSV files
def dump_csv_file(fname, count=5):
    # count: 0 - column names only, -1 - all rows, default = 5 rows max
    df = pd.read_csv(fname)
    if count < 0:
        count = df.shape[0]
    return df.head(count)

########################################################################
# Dataset related functions
ds_nbaiot = '/kaggle/input/nbaiot-dataset'
dn_nbaiot = ['Danmini_Doorbell', 'Ecobee_Thermostat', 'Ennio_Doorbell', 'Philips_B120N10_Baby_Monitor', 'Provision_PT_737E_Security_Camera', 'Provision_PT_838_Security_Camera', 'Samsung_SNH_1011_N_Webcam', 'SimpleHome_XCS7_1002_WHT_Security_Camera', 'SimpleHome_XCS7_1003_WHT_Security_Camera']

def fname(ds, f):
    if '.csv' not in f:
        f = f'{f}.csv'
    return os.path.join(ds, f)

def fname_nbaiot(f):
    return fname(ds_nbaiot, f)

def get_nbaiot_device_files():
    nbaiot_all_files = dump_csv_file(fname_nbaiot('data_summary'), -1)
    nbaiot_all_files = nbaiot_all_files.iloc[:,0:1].values
    device_id = 1
    indices = []
    for j in range(len(nbaiot_all_files)):
        if str(device_id) not in str(nbaiot_all_files[j]):
            indices.append(j)
            device_id += 1
    nbaiot_device_files = np.split(nbaiot_all_files, indices)
    return nbaiot_device_files

def get_nbaiot_device_data(device_id, count_norm=-1, count_anom=-1):
    if device_id < 1 or device_id > 9:
        assert False, "Please provide a valid device ID 1-9, both inclusive"
    if count_anom == -1:
        count_anom = count_norm
    device_index = device_id -1
    device_files = get_nbaiot_device_files()
    device_file = device_files[device_index]
    df = pd.DataFrame()
    y = []
    for i in range(len(device_file)):
        fname = str(device_file[i][0])
        df_c = pd.read_csv(fname_nbaiot(fname))
        count = count_anom
        if 'benign' in fname:
            count = count_norm
        rows = count if count >=0 else df_c.shape[0]
        print("processing", fname, "rows =", rows)
        y_np = np.ones(rows) if 'benign' in fname else np.zeros(rows)
        y.extend(y_np.tolist())
        df = pd.concat([df.iloc[:,:].reset_index(drop=True),
                      df_c.iloc[:rows,:].reset_index(drop=True)], axis=0)
    X = df.iloc[:,:].values
    y = np.array(y)
    Xdf = df
    return (X, y, Xdf)

def get_nbaiot_devices_data():
    devices_data = []
    for i in range(9):
        device_id = i + 1
        (X, y) = get_nbaiot_device_data(device_id)
        devices_data.append((X, y))
    return devices_data
#print_input_files()
print_system_info()

RAM:                   18.621841 GB
CORES:                 4
Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  2
Core(s) per socket:  2
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2200.134
BogoMIPS:            4400.26
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0-3
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx

In [3]:
def gen_teda_obj(observation, k=None, mean=None, var=None, ecc=None):
    teda = {}
    if not k:
        k = 1
        mean = observation
        var = 0
        ecc = 1
    else:
        if mean is None or var is None or ecc is None:
            assert False, 'mean, variance and ecc values are required'

    teda['k'] = k
    teda['observation'] = observation
    teda['mean'] = mean
    teda['var'] = var
    teda['eccentricity'] = ecc
    teda['typicality'] = 1.0 - ecc
    teda['norm_eccentricity'] = teda['eccentricity'] / 2.0
    teda['norm_typicality'] = teda['typicality'] / (k - 2.0)
    teda['outlier'] = 1.0 if teda['norm_eccentricity'] > (1.0 / k) else 0.0
    teda['normal'] = 1.0 if teda['outlier'] < 0.5 else 0.0
    teda['normal_bool'] = True if teda['normal'] > 0.5 else False
    teda['ecc_threshold'] = 1.0 / k

    return teda


def calc_teda_single(observation, teda = None):
    if not teda:
        teda = gen_teda_obj(observation)
    else:
        k = teda['k'] + 1.0
        mean = teda['mean']
        var = teda['var']

        # Calculate the running mean value
        mean =  (((k - 1)  / k) * mean) + ((1 / k) * observation)

        # Calculate the running mean value
        var = (((k - 1) / k) * var) + (1 / (k - 1)) * np.linalg.norm(observation - mean)

        # Calculate the running eccentricity value
        ecc = (1 / k) +  (np.linalg.norm(mean - observation) / (k * var))

        teda = gen_teda_obj(observation, k, mean, var, ecc)

    return teda

def calc_teda(X):
    teda = None
    teda_output = []
    rows = X.shape[0]
    for i in range(rows):
        teda = calc_teda_single(X[i,:], teda)
        teda_output.append(teda['normal'])

    return teda_output

def predict(X, X_norm, X_anom):
    teda_norm = None
    teda_anom = None
    y_pred = []
    rows = X.shape[0]
    rows_norm = X_norm.shape[0]
    rows_anom = X_anom.shape[0]
    for i in range(rows_norm):
        teda_norm = calc_teda_single(X_norm[i,:], teda_norm)

    for i in range(rows_anom):
        teda_anom = calc_teda_single(X_anom[i,:], teda_anom)

    for i in range(rows):
        teda_norm_tmp = calc_teda_single(X[i,:], teda_norm)
        teda_anom_tmp = calc_teda_single(X[i,:], teda_anom)
        
        if (teda_norm_tmp['normal_bool'] == True):
            y_pred.append(1.0)
            teda_norm = teda_norm_tmp
        else:
            y_pred.append(0.0)
            teda_anom = teda_anom_tmp

    return(np.array(y_pred))

def remove_correlated_features(df, threshold):
    df = df.copy()
    # Create correlation matrix
    corr_matrix = df.corr().abs()

    # Select upper triangle of correlation matrix
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))

    # Find features with correlation greater than a threshold
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]

    # Drop features 
    df.drop(to_drop, axis=1, inplace=True)
    return df.iloc[:,:].values

def mark_important_features(vector, pc_keep): # pc_keep is the percentage (0-100) of labels to keep
    th = np.percentile(vector,(100-pc_keep)) # threshold, calculate percentile (100 - percentage) from percentage
    important_bool = (vector >= th)
    important_int = important_bool.astype(int)
    return important_int

def select_features(X, X_norm, X_anom):
    rows_n = X_norm.shape[0]
    rows_a = X_anom.shape[0]
    if rows_n == 0 or rows_a == 0:
        return X

    y_norm = np.ones(rows_n)
    y_anom = np.zeros(rows_a)

    reg_n = LinearRegression(fit_intercept=False)
    reg_n.fit(X_norm, y_norm)
    coef_n = abs(reg_n.coef_)
    n = mark_important_features(coef_n, 85)

    reg_a = LinearRegression(fit_intercept=False)
    reg_a.fit(X_anom, y_anom)
    coef_a = abs(reg_a.coef_)
    a = mark_important_features(coef_a, 85)

    mask = np.bitwise_or(n,a)
    mask = mask == 1 # convert to Boolean
    X_sel = X[:, mask]
    return X_sel

In [4]:
for i in range(9):
    device_index = i
    device_id = device_index + 1
    device_name = dn_nbaiot[device_index]
    (X, y, Xdf) = get_nbaiot_device_data(device_id)
    features_before = X.shape[1]
    X = remove_correlated_features(Xdf, 0.95)
    features_after = X.shape[1]
    X_std = StandardScaler().fit_transform(X)
    indices_norm = np.where(y >= 0.5)
    indices_anom = np.where(y <= 0.5)
    X_norm_all = X_std[indices_norm]
    X_anom_all = X_std[indices_anom]
    step = 0.05
    count_frac = 0.0
    print(device_name)
    print('total features', features_before)
    print('final features', features_after)
    print("features_final,features_selected,count_norm,count_anom,count_frac,acc,tn,fp,fn,tp")
    while count_frac <= 1.01:
        count_norm = int(count_frac * float(X_norm_all.shape[0]))
        count_anom = int(count_frac * float(X_anom_all.shape[0]))
        X_norm = X_norm_all[0:count_norm,:]
        X_anom = X_anom_all[0:count_anom,:]
        X_sel = select_features(X_std, X_norm, X_anom)
        features_final = X_std.shape[1]
        features_selected = X_sel.shape[1]
        y_pred = predict(X_sel, X_norm, X_anom)
        tn, fp, fn, tp = confusion_matrix(y, y_pred, labels=[0,1]).ravel()
        acc = accuracy_score(y, y_pred)
        print(f'{features_final:010d},{features_selected:010d},{count_norm:010d},{count_anom:010d},{count_frac:.2f},{acc:.2f},{tn},{fp},{fn},{tp}')
        count_frac += step

processing 1.benign.csv rows = 49548
processing 1.gafgyt.combo.csv rows = 59718
processing 1.gafgyt.junk.csv rows = 29068
processing 1.gafgyt.scan.csv rows = 29849
processing 1.gafgyt.tcp.csv rows = 92141
processing 1.gafgyt.udp.csv rows = 105874
processing 1.mirai.ack.csv rows = 102195
processing 1.mirai.scan.csv rows = 107685
processing 1.mirai.syn.csv rows = 122573
processing 1.mirai.udp.csv rows = 237665
processing 1.mirai.udpplain.csv rows = 81982
Danmini_Doorbell
total features 115
final features 36
features_final,features_selected,count_norm,count_anom,count_frac,acc,tn,fp,fn,tp


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


0000000036,0000000036,0000000000,0000000000,0.00,0.95,968101,649,49440,108


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


0000000036,0000000036,0000002477,0000048437,0.05,0.98,968633,117,18670,30878


...

0000000045,0000000045,0000017575,0000748168,0.90,0.56,458838,372460,2594,16934


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


0000000045,0000000045,0000018551,0000789733,0.95,0.56,458908,372390,2602,16926


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in double_scalars


0000000045,0000000045,0000019528,0000831298,1.00,0.56,455498,375800,2596,16932
